* Notebook created by nov05 on 2024-12-29
* It was run locally with conda env `awsmle_py310`.  

---   

* [Docs > Models and pre-trained weights > ResNet > resnet34](https://pytorch.org/vision/main/models/generated/torchvision.models.resnet34.html)  

In [6]:
%pwd

'd:\\github\\udacity-nd009t-capstone-starter\\starter'

In [1]:
## windows cmd to launch notepad to edit aws credential file
# !notepad C:\Users\guido\.aws\config
!notepad C:\Users\guido\.aws\credentials

In [9]:
## reset the session after updating credentials
import boto3 # type: ignore
boto3.DEFAULT_SESSION = None
import sagemaker # type: ignore
from sagemaker import get_execution_role # type: ignore

role_arn = get_execution_role()  ## get role ARN
if 'AmazonSageMaker-ExecutionRole' not in role_arn:
    ## Go to "IAM - Roles", search for "SageMaker", find the execution role.
    voclabs_role_arn = role_arn
    sagemaker_role_arn = "arn:aws:iam::852125600954:role/service-role/AmazonSageMaker-ExecutionRole-20241119T203081"
session = sagemaker.Session()  ## "default"
region = session.boto_region_name
bucket = session.default_bucket()
print("AWS Region: {}".format(region))
print("Default Bucket: {}".format(bucket))
print(f"Role voclabs ARN: {voclabs_role_arn}") ## If local, Role ARN: arn:aws:iam::807711953667:role/voclabs
print("SageMaker Role ARN: {}".format(sagemaker_role_arn)) 

import wandb
## generate secrets.env. remember to add it to .gitignore  
wandb.sagemaker_auth(path="../secrets") 

[01/26/25 20:04:47] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=936309;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=468704;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\botocore\credentials.py#1278\1278]8;;\

[01/26/25 20:04:49] WARNING  Couldn't call 'get_role' to get Role ARN from role name voclabs to get ]8;id=596400;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=324330;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\session.py#5902\5902]8;;\
                             Role path.                                                                            

                    INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=285360;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=913261;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\botocore\credentials.py#1278\1278]8;;\

AWS Region: us-east-1
Default Bucket: sagemaker-us-east-1-570668189909
Role voclabs ARN: arn:aws:iam::570668189909:role/voclabs
SageMaker Role ARN: arn:aws:iam::852125600954:role/service-role/AmazonSageMaker-ExecutionRole-20241119T203081


# TODO: Title
**TODO**: Give a helpful introduction to what this notebook is for. Remember that comments, explanations and good documentation make your project informative and professional.

**Note:** This notebook has a bunch of code and markdown cells with TODOs that you have to complete. These are meant to be helpful guidelines for you to finish your project while meeting the requirements in the project rubrics. Feel free to change the order of the TODO's and/or use more than one cell to complete all the tasks.

In [ ]:
# TODO: Install any packages that you might need

In [ ]:
# TODO: Import any packages that you might need

## Data Preparation
**TODO:** Run the cell below to download the data.

The cell below creates a folder called `data`, downloads training data and arranges it in subfolders. Each of these subfolders contain images where the number of objects is equal to the name of the folder. For instance, all images in folder `1` has images with 1 object in them. Images are not divided into training, testing or validation sets. If you feel like the number of samples are not enough, you can always download more data (instructions for that can be found [here](https://registry.opendata.aws/amazon-bin-imagery/)). However, we are not assessing you on the accuracy of your final trained model, but how you create your machine learning engineering pipeline.

* View the S3 bucket in your account   
    https://s3.console.aws.amazon.com/s3/buckets/aft-vbi-pds
    

In [26]:
## example code to download a file from s3 bucket
import boto3
from botocore import UNSIGNED
from botocore.client import Config
# Create an S3 client with unsigned requests (public access)
s3_client = boto3.client('s3', config=Config(signature_version=UNSIGNED))
s3_client.download_file(
    Bucket='aft-vbi-pds',
    Key='bin-images/100313.jpg',
    Filename='../data/bin-images/100313.jpg'
)

In [25]:
import os
import json
from tqdm import tqdm
import boto3
from botocore import UNSIGNED
from botocore.client import Config

def download_and_arrange_data(
        prefix='bin-images', 
        file_extension='.jpg',
        download_dir='../data/bin-images',
        partition=True):
    
    s3_client = boto3.client('s3', config=Config(signature_version=UNSIGNED))  ## public access

    ## There are 140536 image file names in the list. 
    with open('file_list.json', 'r') as f:
        d = json.load(f)

    for k, v in d.items():  ## There are 5 items (for 5 classes) in the JSON file.
        print(f"Downloading images/metadata of images with {k} object...")
        if partition:
            download_dir = os.path.join(download_dir, k)
        if not os.path.exists(download_dir):
            os.makedirs(download_dir)
        for file_path in tqdm(v):
            file_name = os.path.basename(file_path).split('.')[0] + file_extension
            s3_client.download_file(
                'aft-vbi-pds', 
                prefix+'/'+file_name,  ## e.g. metadata/100313.json
                download_dir+'/'+file_name)
            
## download metadata, 17.9 MB, 56m 57.4s
download_and_arrange_data(
    prefix='metadata', 
    file_extension='.json',
    download_dir='../data/metadata',
    partition=False)

100%|██████████| 1228/1228 [06:36<00:00,  3.09it/s]


100%|██████████| 2299/2299 [12:38<00:00,  3.03it/s]


100%|██████████| 2666/2666 [14:35<00:00,  3.04it/s]


100%|██████████| 2373/2373 [12:54<00:00,  3.06it/s]


100%|██████████| 1875/1875 [10:11<00:00,  3.07it/s]


In [1]:
print("total metadata file number:", 1228 + 2299 + 2666 + 2373 + 1875)

total metadata file number: 10441


## Dataset
**TODO:** Explain what dataset you are using for this project. Give a small overview of the classes, class distributions etc that can help anyone not familiar with the dataset get a better understanding of it. You can find more information about the data [here](https://registry.opendata.aws/amazon-bin-imagery/).  

* [ScriptProcessor](https://sagemaker.readthedocs.io/en/stable/api/training/processing.html#sagemaker.processing.ScriptProcessor)  
* [Tutorial](https://docs.google.com/document/d/17KzWVf84xQJVNH1jd6yh_FLgr781QcdKng1JIF6P5X4): Create custom docker image for SageMaker data processing jobs, create AWS ECR private repo, and upload the image to the repo    

In [ ]:
## get my own AWS account number
with open('../secrets/aws_account_number', 'r') as file:
    for line in file:
        aws_account_number = line.strip()
        break

In [ ]:
#TODO: Perform any data cleaning or data preprocessing
from sagemaker.processing import ScriptProcessor
processor = ScriptProcessor(
    command=['python3'],
    image_uri=f'{aws_account_number}.dkr.ecr.us-east-1.amazonaws.com/udacity/p5-amazon-bin-images:latest',  # You can use a custom image if needed
    role=sagemaker_role_arn,  # Execution role
    instance_count=1,
    instance_type='ml.t3.large',  # Use the appropriate instance type
    volume_size_in_gb=10,  # Minimal disk space since we're streaming
    base_job_name='p5-amazon-bin-images' 
)
# Run the processing job
processor.run(
    code='../scripts_process/webdataset.py',  # Your script to process data
    environment={  # Pass environment variables for input/output S3
        'SM_CHANNEL_TRAINING': 's3://your-input-bucket',
        'OUTPUT_S3': 's3://your-output-bucket',
        'PREFIX_IMAGES': 'images/',
        'PREFIX_METADATA': 'metadata/'
    }
)

In [ ]:
from sagemaker.pytorch import PyTorch
from sagemaker.inputs import TrainingInput
data_base_path = "s3://p3-dog-breed-image-classification/dogImages/"
train_data = TrainingInput(data_base_path+"train/", content_type="image/jpeg")
val_data = TrainingInput(data_base_path+"valid/", content_type="image/jpeg")
test_data = TrainingInput(data_base_path+"test/", content_type="image/jpeg")
## ⚠️ don't use prefix in output_path, cause source folder will be created 
## at bucket level, while other folders, e.g. debug-output, at prefix levle.
output_path = "s3://p3-dog-breed-image-classification/"  

## Model Training
**TODO:** This is the part where you can train a model. The type or architecture of the model you use is not important. 

**Note:** You will need to use the `train.py` script to train your model.

In [ ]:
#TODO: Declare your model training hyperparameter.
#NOTE: You do not need to do hyperparameter tuning. You can use fixed hyperparameter values
from sagemaker.debugger import (
    Rule,
    ## debugger
    DebuggerHookConfig,
    rule_configs,
    ## profiler 
    ProfilerRule,
    ProfilerConfig,
    FrameworkProfile
)
hyperparameters = {
    'epochs': 40,   
    'batch-size': 32,   
    'opt-learning-rate': 8e-5,  
    'opt-weight-decay': 1e-5,  
    'lr-sched-step-size': 5,  
    'lr-sched-gamma': 0.5,
    'early-stopping-patience': 5,
    'model-type': 'resnet50', 
    'wandb': True,  
    'debug': True, 
}
rules = [
    Rule.sagemaker(rule_configs.poor_weight_initialization()),
    Rule.sagemaker(rule_configs.vanishing_gradient()),
    Rule.sagemaker(rule_configs.overfit()),
    Rule.sagemaker(rule_configs.overtraining()),
    Rule.sagemaker(rule_configs.loss_not_decreasing()),
    ProfilerRule.sagemaker(rule_configs.LowGPUUtilization()),
    ProfilerRule.sagemaker(rule_configs.ProfilerReport()),
]
hook_config = DebuggerHookConfig(
    hook_parameters={
        "train.save_interval": "100", 
        "eval.save_interval": "10"
    }
)
profiler_config = ProfilerConfig(
    system_monitor_interval_millis=500, 
    framework_profile_params=FrameworkProfile(num_steps=10)
)

In [ ]:
#TODO: Create your training estimator
estimator = PyTorch(
    entry_point='train.py',  # Your training script that defines the ResNet50 model and training loop
    source_dir='scripts_train',  # Directory where your script and dependencies are stored
    role=role_arn,
    framework_version='1.13.1',  # Use the PyTorch version you need
    py_version='py39',
    instance_count=1,  # Adjust based on the number of instances you want to use
    # instance_type='ml.p3.2xlarge',  ## 16GB, Use GPU instances for deep learning
    instance_type='ml.g4dn.xlarge',  ## 16GB
    output_path=output_path,  ## if not specify, output to the sagemaker default bucket
    hyperparameters=hyperparameters,
    # use_spot_instances=True,
    ## Debugger and profiler parameters
    rules=rules,
    debugger_hook_config=hook_config,    
    profiler_config=profiler_config,
)

In [ ]:
%%time
# TODO: Fit your estimator
from datetime import datetime
estimator.fit(
    wait=True,  
    job_name=f"p5-amazon-bin-job-{datetime.now().strftime('%Y%m%d-%H%M%S')}",  
    inputs={
        "train": train_data,  
        "validation": val_data, 
        "test": test_data,
    },  
)

## Standout Suggestions
You do not need to perform the tasks below to finish your project. However, you can attempt these tasks to turn your project into a more advanced portfolio piece.

### Hyperparameter Tuning
**TODO:** Here you can perform hyperparameter tuning to increase the performance of your model. You are encouraged to 
- tune as many hyperparameters as you can to get the best performance from your model
- explain why you chose to tune those particular hyperparameters and the ranges.


In [ ]:
#TODO: Create your hyperparameter search space

In [ ]:
#TODO: Create your training estimator

In [ ]:
# TODO: Fit your estimator

In [ ]:
# TODO: Find the best hyperparameters

### Model Profiling and Debugging
**TODO:** Use model debugging and profiling to better monitor and debug your model training job.

In [ ]:
# TODO: Set up debugging and profiling rules and hooks

In [ ]:
# TODO: Create and fit an estimator

In [ ]:
# TODO: Plot a debugging output.

**TODO**: Is there some anomalous behaviour in your debugging output? If so, what is the error and how will you fix it?  
**TODO**: If not, suppose there was an error. What would that error look like and how would you have fixed it?

In [ ]:
# TODO: Display the profiler output

### Model Deploying and Querying
**TODO:** Can you deploy your model to an endpoint and then query that endpoint to get a result?

In [ ]:
# TODO: Deploy your model to an endpoint

In [ ]:
# TODO: Run an prediction on the endpoint

In [ ]:
# TODO: Remember to shutdown/delete your endpoint once your work is done

### Cheaper Training and Cost Analysis
**TODO:** Can you perform a cost analysis of your system and then use spot instances to lessen your model training cost?

In [ ]:
# TODO: Cost Analysis

In [ ]:
# TODO: Train your model using a spot instance

### Multi-Instance Training
**TODO:** Can you train your model on multiple instances?

In [ ]:
# TODO: Train your model on Multiple Instances